In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,LeakyReLU
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


from mtcnn.mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt
detector = MTCNN()
import cv2

Using TensorFlow backend.


In [2]:
model = Sequential()
#1
model.add(Conv2D(32, kernel_size=(3, 3),input_shape=(48,48,1)))

model.add(LeakyReLU(alpha=0.1))
#2
model.add(Conv2D(32, kernel_size=(3, 3)))

model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.20))
#3
model.add(Conv2D(64, kernel_size=(3, 3)))

model.add(LeakyReLU(alpha=0.1))

#
model.add(Dropout(0.20))



model.add(MaxPooling2D(pool_size=(2, 2)))
#4
model.add(Conv2D(64, kernel_size=(3, 3)))

model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.20))

model.add(Flatten())

model.add(Dense(512))

model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

model.load_weights('model.h5')

In [3]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 
                        3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

emoji_dist={0:"./emojis/chopper/angry.jpg",
            1:"./emojis/chopper/disgusted.jpg",2:"./emojis/chopper/fearful.jpg",
            3:"./emojis/chopper/happy.jpg",
            4:"./emojis/chopper/neutral.jpg",
            5:"./emojis/chopper/sad.jpg",6:"./emojis/chopper/surprised.jpg"}


In [ ]:
rand=cv2.VideoCapture(0)
while True:
    confirm, frame=rand.read()
    #frame[0:250,390::]=0
    if not confirm :
        break
    grayed=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    faces = detector.detect_faces(frame)
    value= None
    for i in faces:
        if len(i["box"])>0:
        
            left,top,right,bottom=i["box"]
    #print(faces)
    #cv2.imshow("frame",frame)
            cv2.rectangle(frame,(left,top),(right+left,bottom+top),(0,0,255),2)
            cropped=grayed[top:top+bottom,left:left+right]
            sample=np.expand_dims(np.expand_dims(cv2.resize(cropped,(48,48)),axis=2),axis=0)
            q=model.predict(sample)
            value=np.argmax(q)
            predicted_emotion=emotion_dict[value]
            
            k=cv2.imread(emoji_dist[value])
            k=cv2.resize(k,(100,100))
            frame[0:100,540::]=0
            frame[0:100,540::]=k
            cv2.putText(frame,predicted_emotion,(left+20,top-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    #frame[0:250,390::]=cv2.resize(cv2.imread(emoji_dist[0],(250,250)))
    cv2.imshow("bro",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()